In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import pyodbc
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
import numpy as np

In [2]:
def database_connection():
    conn = pyodbc.connect(
        r'DRIVER={SQL Server Native Client 11.0};'
        r'SERVER=localhost;'
        r'DATABASE=seis603_pricing_project;'
        r'UID=python_dev;'
        r'PWD=python_dev;'
        )
    cursor = conn.cursor()
    return cursor

db_connection = database_connection()

In [3]:
#items tracking
sql = "Select * from dbo.Baseball_Stats"
db_connection.execute(sql)#, params) #executing sproc
list_items = db_connection.fetchall()#[0] #fetchone will only return first result
#df = pd.Series(list_items)
labels = ['pk','regular_season','post_season','year','league']
df = pd.DataFrame.from_records(list_items, columns=labels) #create dataframe from list
print(df['league'].unique())

['AA' 'NL' 'AL']


In [4]:
#more AA league
df = df[df['league'] != 'AA']
print(df['league'].unique())

year = df["year"].unique()
#converting to floats
df['regular_season'] = df['regular_season'].astype('float')
df['post_season'] = df['post_season'].astype('float')

regular_season = df.groupby('year')['regular_season'].mean()
post_season = df.groupby('year')['post_season'].mean()

print(post_season.head())

['NL' 'AL']
year
1884    1.000000
1885    0.600000
1886    0.500000
1887    0.655556
1888    0.666667
Name: post_season, dtype: float64


In [5]:
regular_trace = go.Scatter(
    x = regular_season.index,#dates
    y = regular_season,
    name = 'Regular Season',
    mode = 'markers',
    line = dict(
        color = ('rgb(49, 86, 219)'),
        width = 4)
)

post_trace = go.Scatter(
    x = post_season.index,#dates
    y = post_season,
    name = 'Post Season',
    mode = 'markers',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

#print(regular_trace)


In [6]:

#all the data i.e. the lines in the graph
data = [regular_trace, post_trace]

#Creating layout of report
layout = go.Layout(
    title='Compare Regular Season Vs Post Season Over the Years',
    xaxis=dict(
        title='Year',
        zeroline = False,
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Ratio',
        zeroline = False,
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-line')